In [ ]:
!git clone https://github.com/Emeika/image-deblur.git

fatal: destination path 'image-deblur' already exists and is not an empty directory.


In [ ]:
cd image-deblur/

[Errno 2] No such file or directory: 'image-deblur/'
/content


***Import Modules***

In [ ]:
import os
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Reshape, Conv2D, UpSampling2D, Input, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img
from sklearn.model_selection import GridSearchCV



!***Extract features into a feature vector and flatten them onto a csv***

In [ ]:
def load_and_preprocess_images(image_folder, label):
    data = []
    for file in tqdm(sorted(os.listdir(image_folder))):
        if any(file.endswith(extension) for extension in ['.jpg', '.JPG']):
            image_path = os.path.join(image_folder, file)
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
            image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
            image_flattened = image.flatten()  # Flatten the image
            image_flattened_str = ','.join(map(str, image_flattened))  # Convert to a single string
            data.append([image_path, image_flattened_str, label])  # Add image path and label
    return data

# Extract features
blur_features = load_and_preprocess_images('/content/image-deblur/resized_dataset/blur', label=0)
sharp_features = load_and_preprocess_images('/content/image-deblur/resized_dataset/sharp', label=1)

# Combine data and create a DataFrame
columns = ['image_path', 'image_features', 'label']
data = sharp_features + blur_features
features_df = pd.DataFrame(data, columns=columns)

# Save to CSV
features_df.to_csv('image_data.csv', index=False)


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tqdm import tqdm

def load_and_preprocess_images(image_folder, label, prefix):
    data = []
    for idx, file in enumerate(tqdm(sorted(os.listdir(image_folder)))):
        if any(file.endswith(extension) for extension in ['.jpg', '.JPG']):
            image_path = os.path.join(image_folder, file)
            image = load_img(image_path, target_size=(128, 128))
            image_array = img_to_array(image).astype('float32') / 255
            image_flattened = image_array.flatten()  # Flatten the image
            image_flattened_str = ','.join(map(str, image_flattened))  # Convert to a single string
            # Use a prefix (e.g., "blur" or "sharp") and an index to pair images
            pair_id = f"{prefix}_{idx}"
            data.append([image_path, image_flattened_str, label, pair_id])  # Add pair_id
    return data

# Extract features
blur_features = load_and_preprocess_images('/content/image-deblur/resized_dataset/blur', label=0, prefix='blur')
sharp_features = load_and_preprocess_images('/content/image-deblur/resized_dataset/sharp', label=1, prefix='sharp')

# Combine data and create a DataFrame
columns = ['image_path', 'image_features', 'label', 'pair_id']
data = sharp_features + blur_features
features_df = pd.DataFrame(data, columns=columns)

# Save to CSV
features_df.to_csv('image_data.csv', index=False)


100%|██████████| 350/350 [00:15<00:00, 22.87it/s]


***Load Data***

In [ ]:
def load_images_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    image_paths = df['image_path'].values
    labels = df['label'].values
    image_data = df['image_features'].apply(lambda x: np.fromstring(x, sep=',')).values
    images = np.array([img.reshape(128, 128, 3) for img in image_data])  # Reshape to original dimensions
    return images, image_paths, labels

# Load images from CSV
X, image_paths, y = load_images_from_csv('image_data.csv')

print(X.shape)  # Should be (num_images, 128, 128, 3)
print(image_paths.shape)  # Should be (num_images,)
print(y.shape)  # Should be (num_images,)


(700, 128, 128, 3)
(700,)
(700,)


In [ ]:
def load_images_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    image_paths = df['image_path'].values
    labels = df['label'].values
    pair_ids = df['pair_id'].values
    image_data = df['image_features'].apply(lambda x: np.fromstring(x, sep=',')).values
    images = np.array([img.reshape(128, 128, 3) for img in image_data])  # Reshape to original dimensions

    # Create dictionaries to store blur and sharp images by pair_id
    blur_images = {}
    sharp_images = {}

    for path, label, pair_id, image in zip(image_paths, labels, pair_ids, images):
        if label == 0:
            blur_images[pair_id] = image
        else:
            sharp_images[pair_id] = image

    # Create lists to store the paired images
    paired_blur_images = []
    paired_sharp_images = []
    paired_image_paths = []
    paired_labels = []

    for pair_id in blur_images.keys():
        if pair_id.replace('blur', 'sharp') in sharp_images:
            paired_blur_images.append(blur_images[pair_id])
            paired_sharp_images.append(sharp_images[pair_id.replace('blur', 'sharp')])
            paired_image_paths.append(image_paths)
            paired_labels.append(labels)

    return (np.array(paired_blur_images), np.array(paired_sharp_images),
            np.array(paired_image_paths), np.array(paired_labels))

# Load images from CSV
blur_images, sharp_images, image_paths, y = load_images_from_csv('image_data.csv')

print(blur_images.shape)  # Should be (num_images, 128, 128, 3)
print(sharp_images.shape)  # Should be (num_images, 128, 128, 3)
print(image_paths.shape)  # Should be (num_images,)
print(y.shape)  # Should be (num_images,)


(350, 128, 128, 3)
(350, 128, 128, 3)
(350, 700)
(350, 700)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

def load_images_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    image_paths = df['image_path'].values
    labels = df['label'].values
    image_data = df['image_features'].apply(lambda x: np.fromstring(x, sep=',')).values
    images = np.array([img.reshape(128, 128, 3) for img in image_data])  # Reshape to original dimensions
    return images, image_paths, labels

# Load images from CSV
images, image_paths, labels = load_images_from_csv('image_data.csv')

# Split into sharp and blur images
sharp_images = images[labels == 1]
blur_images = images[labels == 0]

# Split into train and test sets
split_size = int(len(sharp_images) * 0.8)
x_train, x_test = blur_images[:split_size], blur_images[split_size:]
y_train, y_test = sharp_images[:split_size], sharp_images[split_size:]

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


(280, 128, 128, 3) (280, 128, 128, 3) (70, 128, 128, 3) (70, 128, 128, 3)


***Train Model by Split into Training, Testing and Validation set using kfolds cross validation***

In [ ]:
# Define the number of folds
num_folds = 5

# Initialize the KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store training and validation losses
train_losses = []
val_losses = []
image_shape = (128, 128, 3)

# Iterate over the folds
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    print(f"Training Fold {fold + 1}")

    # Get the data for this fold
    X_train_fold, X_val_fold = X[train_index], X[val_index]

    # Reshape features to match image dimensions
    X_train_fold = X_train_fold.reshape(-1, *image_shape)
    X_val_fold = X_val_fold.reshape(-1, *image_shape)

    # Build the model
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=image_shape),
        Conv2D(64, (3, 3), activation='relu', padding='same', strides=2),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same', strides=2),
        UpSampling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(3, (3, 3), activation='sigmoid', padding='same')
    ])

    model.compile(optimizer=Adam(), loss='mean_squared_error')
    # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # batch size = 32

    # Train the model
    history = model.fit(X_train_fold, X_train_fold, epochs=50, batch_size=16, validation_data=(X_val_fold, X_val_fold))

    # Save training and validation losses
    train_losses.append(history.history['loss'])
    val_losses.append(history.history['val_loss'])

    # Save the model
    model.save(f'deblur_model_fold_{fold}.h5')

# Print average training and validation losses across folds
print(f"Average Training Loss: {np.mean(train_losses, axis=0)}")
print(f"Average Validation Loss: {np.mean(val_losses, axis=0)}")


Training Fold 1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 21s 162ms/step - loss: 0.0544 - val_loss: 0.0136
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0120 - val_loss: 0.0075
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0072 - val_loss: 0.0058
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 0.0060 - val_loss: 0.0049
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.0052 - val_loss: 0.0044
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.0041 - val_loss: 0.0052
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 0.0046 - val_loss: 0.0039
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 0.0034 - val_loss: 0.0037
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0036 - val_loss: 0.0042
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.0041 - val_loss: 0.0038
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0

Training Fold 2
Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 6s 91ms/step - loss: 0.0507 - val_loss: 0.0123
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.0100 - val_loss: 0.0066
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 0.0069 - val_loss: 0.0058
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0055 - val_loss: 0.0046
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 0.0048 - val_loss: 0.0042
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0047 - val_loss: 0.0042
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0045 - val_loss: 0.0037
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0041 - val_loss: 0.0039
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.0040 - val_loss: 0.0035
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/

Training Fold 3
Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 92ms/step - loss: 0.0535 - val_loss: 0.0110
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 0.0108 - val_loss: 0.0065
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0073 - val_loss: 0.0064
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 0.0064 - val_loss: 0.0049
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.0051 - val_loss: 0.0047
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0044 - val_loss: 0.0041
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 0.0045 - val_loss: 0.0037
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0041 - val_loss: 0.0035
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0037 - val_loss: 0.0040
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0038 - val_loss: 0.0041
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/

Training Fold 4
Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 114ms/step - loss: 0.0539 - val_loss: 0.0166
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0136 - val_loss: 0.0080
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0075 - val_loss: 0.0064
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0059 - val_loss: 0.0057
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.0054 - val_loss: 0.0048
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 0.0049 - val_loss: 0.0043
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0041 - val_loss: 0.0041
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 0.0040 - val_loss: 0.0047
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0042 - val_loss: 0.0036
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.0036 - val_loss: 0.0035
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 0.0040 - val_loss: 0.0039
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms

Training Fold 5
Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - loss: 0.0522 - val_loss: 0.0147
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - loss: 0.0118 - val_loss: 0.0081
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 0.0073 - val_loss: 0.0064
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0056 - val_loss: 0.0058
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0056 - val_loss: 0.0063
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 0.0048 - val_loss: 0.0050
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0038 - val_loss: 0.0045
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0035 - val_loss: 0.0062
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0045 - val_loss: 0.0044
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0032 - val_loss: 0.0041
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms

Average Training Loss: [0.0371051  0.00983702 0.00671967 0.00554916 0.00512822 0.00461541
 0.00427863 0.00396144 0.00388019 0.00378087 0.00361185 0.00340499
 0.00347098 0.00319855 0.00312108 0.00287376 0.00273219 0.00251699
 0.00245715 0.00251166 0.00227254 0.00225142 0.00225689 0.00215396
 0.00211606 0.00196169 0.00192106 0.00195707 0.00192045 0.00196607
 0.0017722  0.00168426 0.00170994 0.00161877 0.00165806 0.00166612
 0.00161783 0.00157711 0.00153454 0.00162189 0.00160092 0.00149777
 0.00146549 0.0014615  0.00152213 0.00151836 0.00152714 0.00139376
 0.00135539 0.00131923]
Average Validation Loss: [0.01363924 0.00732789 0.00617947 0.00517804 0.00488968 0.00455077
 0.00403916 0.00404577 0.00430348 0.00384389 0.00363798 0.00330435
 0.00340142 0.00333109 0.0032783  0.00292522 0.00263959 0.00247401
 0.00300473 0.00229932 0.00216268 0.00228771 0.0021741  0.00212987
 0.00204556 0.00187622 0.00189464 0.00204209 0.0022735  0.00179471
 0.0018038  0.00165176 0.00167733 0.001644   0.0017483  0

In [ ]:
# Define U-Net model
def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Encoding path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoding path
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(3, 1, activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

# Compile model
model = unet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(blur_images, sharp_images, batch_size=16, epochs=50, validation_split=0.1)

# Save the model
model.save('image_deblurring_model.h5')


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import clone_model

class KerasClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model_builder, **kwargs):
        self.model_builder = model_builder
        self.kwargs = kwargs

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)
        self.model_ = self.model_builder(**self.kwargs)
        self.model_.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.model_.fit(X, y, epochs=10, batch_size=32, verbose=0)
        return self

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        y_pred = np.argmax(self.model_.predict(X), axis=1)
        return y_pred

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)



# You can now use wrapper with GridSearchCV or any other scikit-learn functionality



# Define U-Net model
def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Encoding path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoding path
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(3, 1, activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

# Initialize k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define parameters for grid search
param_grid = {
    'epochs': [50, 100],
    'batch_size': [16, 32],
    'optimizer': ['adam', 'rmsprop']
}

# Grid search
best_model = GridSearchCV(estimator=unet_model(), param_grid=param_grid, cv=kf, scoring='accuracy', verbose=1, n_jobs=-1)
best_model.fit(X, y)


wrapper = KerasClassifierWrapper(best_model)
# Print best parameters and score
print("Best Parameters: ", best_model.best_params_)
print("Best Score: ", best_model.best_score_)

# Save the best model
best_model.best_estimator_.save('best_deblur_model.h5')


# **Inference**

In [ ]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

model = load_model('/content/image-deblur/image-deblur/models/deblur_model_fold_3.h5')

def deblur_image(img_path, output_path, size=(128, 128)):
    # Load and preprocess the input image
    img = load_img(img_path, target_size=size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict using the model
    deblurred_array = model.predict(img_array)[0]

    # Convert the deblurred array to image and save
    deblurred_img = array_to_img(deblurred_array)

    # Plot both images
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Blur Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(deblurred_img)
    plt.title('Predicted Sharp Image')
    plt.axis('off')

    plt.show()

# Example usage
deblur_image('/content/42_NIKON-D3400-35MM_F.JPG', '/content/out.jpg')


TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 128, 128, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_4'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
# preprocess_save_images.py

import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tqdm import tqdm

def load_and_preprocess_images(image_folder, label, prefix):
    data = []
    for idx, file in enumerate(tqdm(sorted(os.listdir(image_folder)))):
        if any(file.endswith(extension) for extension in ['.jpg', '.JPG', '.jpeg', '.png']):
            image_path = os.path.join(image_folder, file)
            image = load_img(image_path, target_size=(128, 128))
            image_array = img_to_array(image).astype('float32') / 255
            image_flattened = image_array.flatten()  # Flatten the image
            image_flattened_str = ','.join(map(str, image_flattened))  # Convert to a single string
            pair_id = f"{prefix}_{idx}"
            data.append([image_path, image_flattened_str, label, pair_id])  # Add pair_id
    return data

# Extract features
blur_features = load_and_preprocess_images('/content/image-deblur/resized_dataset/blur', label=0, prefix='blur')
sharp_features = load_and_preprocess_images('/content/image-deblur/resized_dataset/sharp', label=1, prefix='sharp')

# Combine data and create a DataFrame
columns = ['image_path', 'image_features', 'label', 'pair_id']
data = sharp_features + blur_features
features_df = pd.DataFrame(data, columns=columns)

# Save to CSV
features_df.to_csv('image_data.csv', index=False)


100%|██████████| 350/350 [00:11<00:00, 30.86it/s]
